In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv
/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
#Lowercasing
#Example 
df['review'][2].lower()

'i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). while some may be disappointed when they realize this is not match point 2: risk addiction, i thought it was proof that woody allen is still fully in control of the style many of us have grown to love.<br /><br />this was the most i\'d laughed at one of woody\'s comedies in years (dare i say a decade?). while i\'ve never been impressed with scarlet johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />this may not be the crown jewel of his career, but it was wittier than "devil wears prada" and more interesting than "superman" a great comedy to go see with friends.'

In [4]:
df['review']=df['review'].str.lower()

In [5]:
#Removing HTml tags 
# Example " but spirited young woman.<br /><br />this may not be t"

import re

def remov_html(text):
    cleaned_text = re.compile('<.*?>').sub(r'', text)
    return cleaned_text

# ans = remov_html("Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.")
# print(ans)


In [6]:
df['review']=df['review'].apply(remov_html)

In [7]:
# t=remov_html('https://www.kaggle.com/oniicha932/<br>text-processing/edit')
# t

In [8]:
import re

def remove_urls(text):
    cleaned_text = re.compile(r'https?://\S+|www\.\S+').sub(r'', text)
    return cleaned_text

text = """There are various types of URLs used on the internet. For example:
- A regular URL: Visit our website at https://www.example.com for more information.
- A secure URL: You can securely access your account at https://secure.example.com.
- A URL with a path: The documentation can be found at https://www.example.com/docs/user-guide.
- A URL with parameters: Search results can be accessed via https://www.example.com/search?q=URL.
- A URL with a port number: Access the application at http://example.com:8080.
- A URL with an anchor: Learn more in our FAQ section: https://www.example.com/faq#section3.
- A mailto URL: Contact us via email at mailto:info@example.com.
- A tel URL: Call our support team at tel:+123456789."""

cleaned_text = remove_urls(text)
print(cleaned_text)


There are various types of URLs used on the internet. For example:
- A regular URL: Visit our website at  for more information.
- A secure URL: You can securely access your account at 
- A URL with a path: The documentation can be found at 
- A URL with parameters: Search results can be accessed via 
- A URL with a port number: Access the application at 
- A URL with an anchor: Learn more in our FAQ section: 
- A mailto URL: Contact us via email at mailto:info@example.com.
- A tel URL: Call our support team at tel:+123456789.


In [9]:
df['review']=df['review'].apply(remove_urls)

In [10]:
# remove puncation
# WHy necesary to remove punctuation?

# TOkenize will treat all puncation as indiviual
# Exmaple : Aron! what did you do?
# TOkenizer: 'Aron', '!','what','did','you','do','?'


In [11]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
remov_p=string.punctuation

In [13]:
# def remove_pun(text):
#     punc = string.punctuation
#     for i in punc:
#         text = text.replace(i, '')
#     return text

# t = remove_pun('Aron! what did you do?')
# print(t)



# #But this is too much time consuming

In [14]:
import string

def remove_pun(text):
    exclude = set(string.punctuation)  # Define the set of punctuation characters
    return text.translate(str.maketrans('', '', ''.join(exclude)))




In [15]:
df['review']=df['review'].apply(remove_pun)

In [16]:
#Chat word treatment 
# Ex: Gn-> Good night
#     Asap-> As soon as possible 

chat_words = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It\'s Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you (also a chat program)',
    'ILU': 'I Love You',
    'IMHO': 'In My Honest/Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My A** Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The A**',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A** Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The F...',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait...',
    '7K': 'Sick:-D Laugher',
    'TFW': 'That feeling when. TFW internet slang often goes in a caption to an image.',
    'MFW': 'My face when',
    'MRW': 'My reaction when',
    'IFYP': 'I feel your pain',
    'TNTL': 'Trying not to laugh',
    'JK': 'Just kidding',
    'IDC': 'I don’t care',
    'ILY': 'I love you',
    'IMU': 'I miss you',
    'ADIH': 'Another day in hell',
    'ZZZ': 'Sleeping, bored, tired',
    'WYWH': 'Wish you were here',
    'TIME': 'Tears in my eyes',
    'BAE': 'Before anyone else',
    'FIMH': 'Forever in my heart',
    'BSAAW': 'Big smile and a wink',
    'BWL': 'Bursting with laughter',
    'LMAO': 'Laughing my a** off',
    'BFF': 'Best friends forever',
    'CSL': 'Can’t stop laughing'
}


def og(text):
    t=text.split()
    for i in t:
        if t.upper in chat_words:
            replace(t.upper,)

In [17]:
chat_words = {
    'gn': 'Good night',
    'asap': 'As soon as possible',
    # Add more slang words and their meanings as needed
}

def replace_slang(text):
    words = text.split()
    for i in words:
        if i.lower() in chat_words:
            print(i)
            i=replace(i,chat_words[i])
    return ' '.join(i)


# sentence="Hey, gn! I need that report asap."
# if 'asap' in sentence:
#     sentence=sentence.replace('asap','As soon as ')
#     print(sentence)
#     print('YEs')
    

    
# Example usage:
sentence = "Hey, gn! I need that report asap."
translated_sentence = replace_slang(sentence)
print(translated_sentence)

a s a p .


In [18]:
chat_words = {
    'gn': 'Good night',
    'asap': 'As soon as possible',
    # Add more slang words and their meanings as needed
}

def replace_slang(text):
    words = text.split()
    print(words)
    for i in words:
        if i.lower() in chat_words.keys():
            print('Yes',i)
    return ' '.join(words)

# Example usage:
sentence = "Hey, gn! I need that report asap."
translated_sentence = replace_slang(sentence)
print(translated_sentence)


['Hey,', 'gn!', 'I', 'need', 'that', 'report', 'asap.']
Hey, gn! I need that report asap.


In [19]:
import re

chat_words = {"gn": "good night", "asap": "as soon as possible", "g8": "great"}
text = "Hey, gn! I need that G8 report asap."

patterns = {key: r"\b{}\b".format(key, "\\b") for key in chat_words}  # Compile regular expressions once

def process_text(text):
    for key, value in chat_words.items():
        text = re.sub(patterns[key], value, text, flags=re.IGNORECASE)
    return text

new_text = process_text(text)
print(new_text)


Hey, good night! I need that great report as soon as possible.


In [20]:
#Spell checkEx:
# Ex:note book , notebook

#     Pyspellchecker 
#     snapcy
#     nltk
    
from textblob import TextBlob
incorrect_txt = 'Sevral deer were huntde by tiger in the forsts. Hunters hunted the animls to svae their cttle'
textblb = TextBlob(incorrect_txt)
corrected_text = textblb.correct()
print(corrected_text)




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Several deer were hunted by tiger in the forests. Hunters hunted the animals to save their cattle


In [21]:
#Remove stop words: Words which are used for sentence formation Example :a, are ,my the 

#IMP in case using POS tagging , stop words should not be removed

from nltk.corpus import stopwords
stopwords.words('english')
                

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [22]:
def remove_stopwrd(text):
    n_txt=[]
    for wrds in text.split():
        if wrds in stopwords.words('english'):
            n_txt.append('')
        else:
            n_txt.append(wrds)
    x=n_txt[:]
    n_txt.clear()
    return " ".join(x)

In [23]:
remove_stopwrd('i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy the plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer while some may be disappointed when they realize this is not match point 2 risk addiction i thought it was proof that woody allen is still fully in control of the style many of us have grown to lovethis was the most id laughed at one of woodys comedies in years dare i say a decade while ive never been impressed with scarlet johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanthis may not be the crown jewel of his career but it was wittier than devil wears prada and more interesting than superman a great comedy to go see with friends')

' thought    wonderful way  spend time    hot summer weekend sitting   air conditioned theater  watching  lighthearted comedy  plot  simplistic   dialogue  witty   characters  likable even  well bread suspected serial killer   may  disappointed   realize    match point 2 risk addiction  thought   proof  woody allen  still fully  control   style many  us  grown  lovethis    id laughed  one  woodys comedies  years dare  say  decade  ive never  impressed  scarlet johanson    managed  tone   sexy image  jumped right   average  spirited young womanthis may    crown jewel   career    wittier  devil wears prada   interesting  superman  great comedy  go see  friends'

In [24]:
df['review'].apply(remove_stopwrd)

0        one    reviewers  mentioned   watching  1 oz e...
1         wonderful little production  filming techniqu...
2         thought    wonderful way  spend time    hot s...
3        basically theres  family   little boy jake thi...
4        petter matteis love   time  money   visually s...
                               ...                        
49995     thought  movie    right good job  wasnt  crea...
49996    bad plot bad dialogue bad acting idiotic direc...
49997       catholic taught  parochial elementary schoo...
49998    im going    disagree   previous comment  side ...
49999     one expects  star trek movies   high art   fa...
Name: review, Length: 50000, dtype: object

In [25]:
#Handling Emojis 😀👋 
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

text_with_emojis = "Hello! 😀👋 This is a sample text with emojis! 🌟🎉"
text_without_emojis = remove_emojis(text_with_emojis)
print(text_without_emojis)


Hello!  This is a sample text with emojis! 


In [26]:
df['review'].apply(remove_emojis)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production the filming tech...
2        i thought this was a wonderful way to spend ti...
3        basically theres a family where a little boy j...
4        petter matteis love in the time of money is a ...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    i am a catholic taught in parochial elementary...
49998    im going to have to disagree with the previous...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [27]:
#Tokenization
#breaking text into smaller parts, it can be words, sentence , phrases
#M1 split function
txt=' i am from Arizona'
txt.split()

['i', 'am', 'from', 'Arizona']

In [28]:
txt='I am moving to California, Since I got job offer their'
txt.split(',')

['I am moving to California', ' Since I got job offer their']

In [29]:
#Issue with split function
txt="When is the birthday party? I need to get the gifts"
txt.split(',')

['When is the birthday party? I need to get the gifts']

In [30]:
#  M2  REgular Expression
import re
txt="When is the birthday party? I need to get the gifts"
tokens=re.findall("[\w']+",txt)
tokens

['When',
 'is',
 'the',
 'birthday',
 'party',
 'I',
 'need',
 'to',
 'get',
 'the',
 'gifts']

In [31]:
#M3 use nltk tokenize

from nltk.tokenize import word_tokenize, sent_tokenize
txt="Natural language processing (NLP) is a field of artificial intelligence that helps computers understand, interpret, and manipulate human language. It aims to enable computers to comprehend the meaning behind texts and to generate human-like responses. Sentiment analysis, language translation, and chatbot development are some applications of NLP. NLTK (Natural Language Toolkit) is a popular library in Python used for NLP tasks."
word_tokenize(txt)

['Natural',
 'language',
 'processing',
 '(',
 'NLP',
 ')',
 'is',
 'a',
 'field',
 'of',
 'artificial',
 'intelligence',
 'that',
 'helps',
 'computers',
 'understand',
 ',',
 'interpret',
 ',',
 'and',
 'manipulate',
 'human',
 'language',
 '.',
 'It',
 'aims',
 'to',
 'enable',
 'computers',
 'to',
 'comprehend',
 'the',
 'meaning',
 'behind',
 'texts',
 'and',
 'to',
 'generate',
 'human-like',
 'responses',
 '.',
 'Sentiment',
 'analysis',
 ',',
 'language',
 'translation',
 ',',
 'and',
 'chatbot',
 'development',
 'are',
 'some',
 'applications',
 'of',
 'NLP',
 '.',
 'NLTK',
 '(',
 'Natural',
 'Language',
 'Toolkit',
 ')',
 'is',
 'a',
 'popular',
 'library',
 'in',
 'Python',
 'used',
 'for',
 'NLP',
 'tasks',
 '.']

In [32]:
sent_tokenize(txt)

['Natural language processing (NLP) is a field of artificial intelligence that helps computers understand, interpret, and manipulate human language.',
 'It aims to enable computers to comprehend the meaning behind texts and to generate human-like responses.',
 'Sentiment analysis, language translation, and chatbot development are some applications of NLP.',
 'NLTK (Natural Language Toolkit) is a popular library in Python used for NLP tasks.']

In [33]:
# M4  spacy 
import spacy
nlp=spacy.load('en_core_web_sm')
doc1=nlp(txt)
for i in doc1:
    print(i)


Natural
language
processing
(
NLP
)
is
a
field
of
artificial
intelligence
that
helps
computers
understand
,
interpret
,
and
manipulate
human
language
.
It
aims
to
enable
computers
to
comprehend
the
meaning
behind
texts
and
to
generate
human
-
like
responses
.
Sentiment
analysis
,
language
translation
,
and
chatbot
development
are
some
applications
of
NLP
.
NLTK
(
Natural
Language
Toolkit
)
is
a
popular
library
in
Python
used
for
NLP
tasks
.


In [34]:
#spacy are quite better result


In [35]:
#Stemming (used for information retrival)
# a natural language processing technique that lowers inflection in words to their root forms
# Ex: Walking-> walk
#     LIKIng  , liked,-> like

# Porter vs. Lancaster:

# Porter tends to be less aggressive and usually produces stems that are more readable and closer to the actual root form of the word. It's a good choice if you're concerned about interpretability and maintaining word meaning to some extent.
# Lancaster is more aggressive, often resulting in shorter stems but might sacrifice readability. It can be beneficial if you need very aggressive stemming for tasks like search indexing where precision might be less critical than reducing words to their simplest forms.
# Snowball (Porter2):

# Snowball stemmers are improvements upon the original Porter stemmer and offer more language support. They have been designed to be more efficient and can handle multiple languages effectively.

In [36]:
from nltk.stem import LancasterStemmer

ls = LancasterStemmer()

def stem_words(text):
    stemmed_words = [ls.stem(word) for word in text.split()]
    return ' '.join(stemmed_words)


In [37]:
stem_words(txt)

'nat langu process (nlp) is a field of art intellig that help comput understand, interpret, and manip hum language. it aim to en comput to comprehend the mean behind text and to gen human-like responses. senty analysis, langu translation, and chatbot develop ar som apply of nlp. nltk (natural langu toolkit) is a popul libr in python us for nlp tasks.'

In [38]:
from nltk.stem import PorterStemmer

ls = PorterStemmer()

def stem_words(text):
    stemmed_words = [ls.stem(word) for word in text.split()]
    return ' '.join(stemmed_words)


In [39]:
stem_words(txt)

'natur languag process (nlp) is a field of artifici intellig that help comput understand, interpret, and manipul human language. It aim to enabl comput to comprehend the mean behind text and to gener human-lik responses. sentiment analysis, languag translation, and chatbot develop are some applic of nlp. nltk (natur languag toolkit) is a popular librari in python use for nlp tasks.'

In [40]:
#some time words are mapped to certain root words which are not actual words 
#ex: nautal : natur

In [41]:
#lemmatization (slow since searching ) use actual valid word for mapping
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer=WordNetLemmatizer()

punctuations="!?:,.;"
sentence_words=nltk.word_tokenize(txt)
for i in sentence_words:
     if i in punctuations:
            sentence_words.remove(i)
sentence_words

['Natural',
 'language',
 'processing',
 '(',
 'NLP',
 ')',
 'is',
 'a',
 'field',
 'of',
 'artificial',
 'intelligence',
 'that',
 'helps',
 'computers',
 'understand',
 'interpret',
 'and',
 'manipulate',
 'human',
 'language',
 'It',
 'aims',
 'to',
 'enable',
 'computers',
 'to',
 'comprehend',
 'the',
 'meaning',
 'behind',
 'texts',
 'and',
 'to',
 'generate',
 'human-like',
 'responses',
 'Sentiment',
 'analysis',
 'language',
 'translation',
 'and',
 'chatbot',
 'development',
 'are',
 'some',
 'applications',
 'of',
 'NLP',
 'NLTK',
 '(',
 'Natural',
 'Language',
 'Toolkit',
 ')',
 'is',
 'a',
 'popular',
 'library',
 'in',
 'Python',
 'used',
 'for',
 'NLP',
 'tasks']